In [2]:
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import train_test_split
from time import time
import pandas as pd 
import numpy as np
from scipy import stats
import nilearn
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from igraph import *


def match(C):
    """
    Replication of R's match function
    """
    
    u, ui = np.unique(C, return_index=True)
    ui = sorted(ui)
    newC = np.zeros(C.shape)
    for i in range(len(ui)):
        newC[np.where(C == C[ui[i]])[0]] = i + 1
        
    return newC


def outer_equal(x):
    """
    Replication of R's outer function with FUN param '=='
    """
    
    res = np.zeros((x.shape[0], x.shape[0]))
    for i in range(x.shape[0]):
        res[i,:] = x == x[i]
    
    return res


def hadamard(nearest):
    """
    Transform data into Hadamard distance matrix
    """
    
    common = nearest.T @ nearest
    ranks = np.outer(np.diag(common), np.ones(nearest.shape[0]))
    neighborUnion = ranks + ranks.T - common
    G = common / neighborUnion
    np.fill_diagonal(G, 0)
    
    return G


def topMax(x, N):
    """
    find Nth largest number in an array
    """
    
    L = len(x)
    assert N < L, 'Number of neighbors cannot be larger than length of data'
    
    while L != 1:
        initial_guess = x[0]
        top_list = x[x > initial_guess]
        bottom_list = x[x < initial_guess]
        
        topL = len(top_list)
        bottomL = len(bottom_list)
        
        if (topL < N) and (L - bottomL >= N):
            x = initial_guess
            break
        
        if topL >= N:
            x = top_list
        else:
            x = bottom_list
            N = N - L + bottomL
        L = len(x)
    
    return x


def bottomMin(x, N):
    """
    find Nth smallest number in an array
    """
    
    return np.round(topMax(x, len(x) - N + 1))


def find_neighbors(D, k):
    """
    Tranform distance matrix to binary k-nearest neighbors graph
    """
    
    nearest = np.zeros(D.shape)
    for i in range(nearest.shape[1]):
        nearest[:,i] = np.round(D[:,i]) <= bottomMin(np.round(D[:,i]), k)
    
    return nearest


def modularity(G, C):
    """
    Calculate graph's modularity
    """
    
    m = np.sum(G)
    Ki = np.repeat(np.sum(G, axis=1), G.shape[1]).reshape(G.shape)
    Kj = Ki.T
    delta_function = outer_equal(C)
    Q = (G - Ki * Kj / m) * delta_function / m
    return np.sum(Q)


def delta_modularity(G, C, i, m, Kj, newC=None):
    """
    Calculate change in modularity by adding a node to a cluster or by removing it
    """
    
    c = np.copy(C)
    
    if newC is None: # removing a node from C
        # removing a solitary node from a cluster does not change the modularity
        if np.sum(c == c[i]) == 1:
            return 0
        newC = c[i]
        c[i] = np.max(c) + 1
        
    I = c == newC
    Ki = np.sum(G[i,:]) # the sum of all the edges connected to node i
    Ki_in = np.sum(G[i,I]) # the sum of all the edges conneting node i to C
    Kjs = np.sum(Kj[I])
    deltaQ = Ki_in / m - Ki * Kjs / m**2
    
    return deltaQ * 2


def louvain_step(G, C, O, Q=None):
    """
    Run a single step of the Louvain algorithm
    """
    
    if not Q:
        Q = modularity(G, C)
        
    m = np.sum(G)
    Kj = np.sum(G, axis=1)
    
    for i in O:
        reassign = np.array(list(set(C[G[i,:] > 0]).difference(set([C[i]]))))
        
        if len(reassign) != 0:
            delta_remove = delta_modularity(G, C, i, m, Kj)

            deltaQ = np.array([delta_modularity(G, C, i, m, Kj, newC=x) for x in reassign])

            if np.max(deltaQ) - delta_remove > 0:
                idx = np.argmax(deltaQ)
                Q += np.max(deltaQ) - delta_remove
                if reassign[idx] > 2389:
                    print(i)
                C[i] = reassign[idx]
            if delta_remove < 0:
                C[i] = np.max(C) + 1
                Q -= delta_remove
    
    return C, Q


def louvain(G, C, maxreps=100):
    """
    Run Louvain algorithm
    """
    
    assert np.allclose(G, G.T), 'Input graph must be symmetric'
    
    order = np.random.permutation(G.shape[1])
    Q = modularity(G, C)
    
    for i in range(maxreps):
        C, Q = louvain_step(G, C, order, Q)
        C = match(C)
        
        # run the second phase, trying to combine each cluster
        n_clust = len(np.unique(C))
        metaG = np.zeros((n_clust, n_clust))
        for ci in range(1, n_clust + 1):
            G_tmp = G[C == ci]
            for cj in range(ci, n_clust + 1):
                G_tmp2 = G_tmp[:,C == cj]
                metaG[ci-1,cj-1] = np.sum(G_tmp2)
                metaG[cj-1,ci-1] = metaG[ci-1,cj-1]
                
        metaC, metaQ = louvain_step(metaG, np.arange(n_clust) + 1, np.random.permutation(n_clust))
        if metaQ - Q > 1e-15:
            tempC = np.copy(C)
            for ci in range(1, n_clust + 1):
                tempC[C == ci] = metaC[ci-1]
            C = match(tempC)
            Q = metaQ
        else:
            break
            
    return C, Q


def plot_community_profiles(X, communities, outfile=None):
    """
    Create radar plot of average profiles of each detected community
    """
    
    # number of variables
    categories = subset
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    fig=plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, polar=True)

    # If you want the first axis to be on top:
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories)
    pi = np.pi
    for label,i in zip(ax.get_xticklabels(), range(0,len(angles))):
        angle_rad = angles[i]
        if angle_rad <= pi/2:
            ha = 'left'
            va = "bottom"
            angle_text = angle_rad * (-180/pi) + 90
        elif pi/2 < angle_rad <= pi:
            ha = 'left'
            va = "top"
            angle_text = angle_rad * (-180/pi) + 90
        elif pi < angle_rad <= (3*pi/2):
            ha = 'right'
            va = "top"  
            angle_text = angle_rad * (-180/pi) - 90
        else:
            ha = 'right'
            va = "bottom"
            angle_text = angle_rad * (-180/pi) - 90
        label.set_rotation(angle_text)
        label.set_verticalalignment(va)
        label.set_horizontalalignment(ha)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks(np.linspace(50,70,5), color="k", size=10)
    plt.ylim(50,70)

    # ------- PART 2: Add plots
    comms, counts = np.unique(communities, return_counts=True)
    for c, n in zip(comms, counts):
        comm_X = X[communities == c,:]
        comm_avg = np.mean(comm_X, axis=0)
        values = comm_avg.tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1.5, linestyle='solid', label="Community %d (N = %d)" % (c, n))
#         ax.fill(angles, values, 'b', alpha=0.1)
    plt.legend(bbox_to_anchor=(1.1, 1.1), fontsize=10)
    if outfile:
        plt.savefig(outfile, bbox_inches='tight', dpi=300)
        
        
def pheno_clust(filepath=None, subset=None, X=None, plot=True, outfile=None, repeats=50, verbose=True):
    """
    Run entire phenoClust pipeline
    """
    
    assert filepath or X is not None, 'Either a matrix or a filepath to the data must be passed in'
    
    if verbose:
        tic = time()
    
    if filepath:
        assert subset, 'If filepath to dataframe is passed, you must also include the subset of columns that you intend to pass into the algorithm'
        
        df = pd.read_csv(filepath)
        df = df[subset]
        df.dropna(inplace=True)

        X = np.array(df)
        
    X_Z = (X - np.mean(X, axis=0, keepdims=True)) / np.std(X, axis=0, dtype=np.float64, ddof=1, keepdims=True)
    D, rho = spearmanr(X_Z, axis=1)
    D = np.round((X_Z.shape[1]**3 - X_Z.shape[1]) * (1 - D) / 6).astype(np.int)
    
    b = np.ceil(np.log2(X_Z.shape[0]) + 1)
    k = np.ceil(X_Z.shape[0] / b).astype(np.int)
    nearest = find_neighbors(D, k + 1)
    G = hadamard(nearest)
    C = np.arange(G.shape[1]) + 1
    
    Q = -np.inf
    for t in range(repeats):
        C, newQ = louvain(G, C)
        if newQ > Q:
            Q = newQ
            if verbose:
                print('iter: %d/%d; Q = %.5f; # of communities: %d' % (t+1, repeats, Q, len(np.unique(C))))
        else:
            break
    
    if verbose:
        toc = time()
        print('PyPhenoClust completed in: %.2f seconds' % (toc - tic))
    
    if plot:
        plot_community_profiles(X, C, outfile)
    
    return C, Q

def bagging_adjacency_matrixes(csv_folder, split):
    '''csv_folder: folder that contains all the csv output from python louvain.
       split: string in the csv files' names that specifies the split eg.Split_1 '''
    
    my_data = [pd.read_csv(csv_folder+file,index_col=0) for file in os.listdir(csv_folder) if split in file] # list contains all the csv in dataframe format
    
    for i in range(len(my_data)): # loop over my_data
        boot=my_data[i].drop_duplicates() #remove duplicates
        df_merge = boot.merge(boot, on='cluster') # create adjacency matrix step1 
        this_adj =pd.crosstab(df_merge.studyid_x, df_merge.studyid_y) # create adjacency matrix step2
        np.fill_diagonal(this_adj.values, 0) #set the diagonal of the adjacency matrix to 0
        
        if i ==0:
            adj_full=this_adj #if this is the first adjacency matrix set it as the initial adj_full 
        else:
            adj_full=adj_full.add(this_adj,fill_value=0) # add this adjacency matrix to the full adjacency matrix


        boot_mask=my_data[i].drop_duplicates()
        boot_mask.cluster=boot_mask.cluster.replace(boot_mask.cluster.unique(),np.ones(boot_mask.cluster.unique().shape))#replace all clusters with 1
        df_merge2 = boot_mask.merge(boot_mask, on='cluster')
        mask_adj =pd.crosstab(df_merge2.studyid_x, df_merge2.studyid_y)
        np.fill_diagonal(mask_adj.values, 0) #set the diagonal of the adjacency matrix to 0
        if i ==0:
            mask_full= mask_adj  #if this is the first adjacency matrix set it as the initial mask_full 
        else:
            mask_full=mask_full.add( mask_adj,fill_value=0) # add this mask adjacency matrix to the full mask adjacency matrix
    
    
    stab_full = adj_full.div(mask_full)
    np.fill_diagonal(stab_full.values, 0)
        
    return adj_full, mask_full, stab_full


In [ ]:
#Subset Variables will depend on the dateset of interest.

##Life Changes Adult Parent: 
#subset = ["R_positivechange","R_inpersonconvo_bin","R_timeoutside","R_restrictionsstress","R_contactschanged",
#          "R_difficultydistancing","R_familychange","R_familychangestress", "R_friendschange","R_friendchangestress",
#          "R_difficultycancellations","R_financedifficulty","R_livingdifficulty","R_foodsecurity"]  #Age and Sex Residuals

#Prior Habits Adult: 
#subset = ["R_bedtimeweekdays", "R_bedtimeweekends","R_hoursofsleepweekdays", "R_hoursofsleepweekends",
#          "R_exerciseprior","R_outdoorsprior","R_priortvmedia","R_priorsocialmedia","R_priorvideogames",
#          "R_threemonthsalcohol", "R_threemonthsvaping","R_threemonthstobacco","R_threemonthsmarijuana", 
#          "R_threemonthsopiates", "R_threemonthsother", "R_threemonthssleepingmeds"] 

#Prior Habits Parent: 
# subset = ["R_priorweekdaybedtime","R_priorweekendbedtime","R_priorhoursofsleepweekdays","R_priorhoursofsleepweekend" ,
#           "R_exerciseprior","R_outdoorsprior", "R_priortvmedia","R_priorsocialmedia", "R_priorvideogames", 
#           "R_threemonthsalcohol", "R_threemonthsvaping", "R_threemonthstobacco", "R_threemonthsmarijuana", "R_threemonthsopiates", "R_threemonthsother", "R_threemonthssleepingmeds"]

In [ ]:
#Pull in data frame (CBCL.CSV) that contains subject ID, CBCL Scores, Age, and Sex
df = pd.read_csv('C:/Users/jacob.derosa/Documents/Combined_Clustering/Clustering_Splits/ADULT_US_C_lf.csv') #Change file path depending on dataset 
df = df.rename(columns={'Unnamed: 0': 'Key'})

#change subset variables depending on analysis
subset = ["R_positivechange","R_inpersonconvo_bin","R_timeoutside","R_restrictionsstress","R_contactschanged","R_difficultydistancing","R_familychange","R_familychangestress",
          "R_friendschange","R_friendchangestress","R_difficultycancellations","R_financedifficulty","R_livingdifficulty","R_foodsecurity"]  #Age and Sex Residuals 
y = np.array(df['Key'])
n_straps = 10000 #number of bootsraps 
n = df.shape[0]
b_idx = np.zeros((n_straps, n))

# bootstrapping
for i in range(n_straps):
    random_state = np.random.RandomState()
    b_idx[i] = random_state.randint(0, high=n - 1, size=n)

b_idx = b_idx.astype(np.int)
y_boot = np.zeros(b_idx.shape, dtype='object')
for i in range(b_idx.shape[0]):
    y_boot[i] = y[b_idx[i]]

bootstrap_split_subids = []
bootstrap_split_communities = []
bootstrap_split_Q = []

for i in range(n_straps):
    X_split = df.iloc[b_idx[i],:]

    bootstrap_split_subids.append([y_boot[i]])
    
    print('bootstrap #%d' % (i + 1), end='\t')
    communities, Q = pheno_clust(X=np.array(X_split[subset]).astype(np.float64), plot=False, verbose=False)
    
    bootstrap_split_communities.append([communities])
    bootstrap_split_Q.append([Q])

allcsvs = {}
for i in range(n_straps):
    df2 = {'Key':bootstrap_split_subids[i][0],'cluster':bootstrap_split_communities[i][0]}
    allcsvs[i] = pd.DataFrame(df2)
    allcsvs[i].to_csv('C:/Users/jacob.derosa/Documents/Combined_Clustering/Clusters/Adult/Data/Adult_US_%s.csv' % i) #change filepath depending on dataset/analysis
    
    
csv_folder= 'C:/Users/jacob.derosa/Documents/Combined_Clustering/Clusters/Adult/Data/' #change filepath depending on dataset/analysis

# Apply Bagging on Split 1 
split = 'Adult_US' #Name of saved CSVs 
adj_full, mask_full, stab_full = bagging_adjacency_matrixes(csv_folder, split)

#Create df Id from matrix column names
columnsNamesArr = pd.DataFrame(stab_full.columns.values)

# Final Louvain Clustering Split 1 
graph = Graph.Weighted_Adjacency(stab_full.values.tolist(), mode=ADJ_UNDIRECTED, attr="weight")
Louvain = graph.community_multilevel(weights=graph.es['weight'])

#Louvain.membership 
Q = graph.modularity(Louvain, weights=graph.es['weight'])
print(Q)

# Create dataframe of Subtypes for Split 1 
Subtypes = pd.DataFrame(Louvain.membership)
Subtypes = Subtypes + 1 

# create dataframe of Split 1 Subtypes 
subs = pd.concat([columnsNamesArr.reset_index(drop=True), Subtypes], axis=1)
subs.columns = ['studyid', 'Subtype']

# read in Split 1 dataframe 
df = pd.read_csv('C:/Users/jacob.derosa/Documents/Combined_Clustering/Clustering_Splits/ADULT_US_C_lf.csv') # Change filepath depending on dataset/analysis
# create Split 1 
Split_1 = pd.merge(subs, df, on='Key')
Split_1.to_csv('C:/Users/jacob.derosa/Documents/Combined_Clustering/Created_Clusters/Adult_UK_C_ls.csv') # Save new path